## 8.2 异步计算

In [1]:
from mxnet import autograd, gluon, nd
from mxnet.gluon import loss as gloss, nn
import os
import subprocess 
import time

### 8.2.1 MXNet中的异步计算

广义上讲，MXNet包括用户直接用来交互的前端和系统用来执行计算的后端。无论使用何种前端编程语言，MXNet程序的执行主要都发生在C++实现的后端。换句话说，用户写好的前端MXNet程序会传给后端执行计算。后端有自己的线程在队列中不断收集任务并执行它们。

MXNet通过前端线程和后端线程的交互实现异步计算。异步计算指，前端线程无需等待当前指令从后端线程返回结果就继续执行后面的指令。

In [2]:
a = nd.ones((1, 2))
b = nd.ones((1, 2))
c = a * b + 2
c


[[3. 3.]]
<NDArray 1x2 @cpu(0)>

In [7]:
class Benchmark(): # 本类已保存在d2lzh包中方便以后使用
    def __init__(self, prefix=None):
        self.prefix = prefix + ' ' if prefix else ''
        
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self, *args):
        print('%stime: %.4f sec' % (self.prefix, time.time() - self.start))

In [8]:
with Benchmark('Workloads are queued.'):
    x = nd.random_uniform(shape=(2000, 2000))
    y = nd.dot(x, x).sum()
    
with Benchmark('Workloads are finished.'):
    print('sum=', y)

Workloads are queued. time: 0.0020 sec
sum= 
[1.9995054e+09]
<NDArray 1 @cpu(0)>
Workloads are finished. time: 0.2015 sec


### 8.2.2 用同步函数让前端等待计算结果

我们可以使用wait_to_read函数让前端等待某个NDArray的计算结果完成，再执行前端中后面的语句。或者，我们可以用waitall函数令前端等待前面所有计算结果完成。

In [9]:
with Benchmark():
    y = nd.dot(x, x)
    y.wait_to_read()

time: 0.1360 sec


In [10]:
with Benchmark():
    y = nd.dot(x, x)
    z = nd.dot(x, x)
    nd.waitall()

time: 0.2432 sec


此外，任何将NDArray转换成其他不支持异步计算的数据结构的操作都会让前端等待计算结果。

In [14]:
with Benchmark():
    y = nd.dot(x, x)
    y.asnumpy()

time: 0.1207 sec


In [15]:
with Benchmark():
    y = nd.dot(x, x)
    y.norm().asscalar()

time: 0.1785 sec


上面介绍的wait_to_read函数、waitall函数、asnumpy函数、asscalar函数和print函数会触发让前端等待后端计算结果的行为。这类函数通常称为同步函数。

### 8.2.3 使用异步计算提升计算性能

In [19]:
with Benchmark('synchronous.'):
    for _ in range(1000):
        y = x + 1
        y.wait_to_read()

synchronous. time: 4.1742 sec


In [20]:
with Benchmark('synchronous.'):
    for _ in range(1000):
        y = x + 1
    nd.waitall()

synchronous. time: 4.5557 sec


我们观察到，使用异步计算能提升一定的计算性能。为了解释这一现象，让我们对Python前端线程和C++后端线程的交互稍作简化。在每一次循环中，前端和后端的交互大约可以分为3个阶段：

1. 前端令后端将计算任务`y = x + 1`放进队列；
1. 后端从队列中获取计算任务并执行真正的计算；
1. 后端将计算结果返回给前端。

我们将这3个阶段的耗时分别设为$t_1, t_2, t_3$。如果不使用异步计算，执行1000次计算的总耗时大约为$1000 (t_1+ t_2 + t_3)$；如果使用异步计算，由于每次循环中前端都无须等待后端返回计算结果，执行1000次计算的总耗时可以降为$t_1 + 1000 t_2 + t_3$（假设$1000t_2 > 999t_1$）。

### 8.2.4 异步计算对内存的影响

我们通常会在每个小批量上评测一下模型，如模型的损失或者准确率。这类评测常用到同步函数，如asscalar函数或者asnumpy函数。如果去掉这些同步函数，前端会将大量的小批量计算任务在极短的时间内丢给后端，从而可能导致占用更多内存。当我们在每个小批量上都使用同步函数时，前端在每次迭代时仅会将一个小批量的任务丢给后端执行计算，并通常会减小内存占用。

In [21]:
def data_iter():
    start = time.time()
    num_batches, batch_size = 100, 1024
    for i in range(num_batches):
        X = nd.random_normal(shape=(batch_size, 512))
        y = nd.ones((batch_size,))
        yield X, y
        if (i + 1) % 50 == 0:
            print('batch %d, time %f sec' % (i + 1, time.time() - start))

In [22]:
net = nn.Sequential()
net.add(nn.Dense(2048, activation='relu'),
        nn.Dense(512, activation='relu'),
        nn.Dense(1))
net.initialize()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.005})
loss = gloss.L2Loss()

In [23]:
def get_mem():
    res = subprocess.check_output(['ps', 'u', '-p', str(os.getpid())])
    return int(str(res).split()[15]) / 1e3

In [24]:
for X, y in data_iter():
    break
loss(y, net(X)).wait_to_read()

In [25]:
l_sum, men = 0, get_mem()
for X, y in data_iter():
    with autograd.record():
        l = loss(y,net(X))
    l_sum += l.mean().asscalar() # 使用同步函数asscalar
    l.backward
    trainer.step(X.shape[0])
nd.waitall()
print('increased memory: %f MB' % (get_mem() - men))

FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [26]:
mem = get_mem()
for X, y in data_iter():
    with autograd.record():
        l = loss(y,net(X))
    l.backward()
    trainer.step(X.shape[0])
nd.waitall()
print('increased memory: %f MB' % (get_mem() - mem))

FileNotFoundError: [WinError 2] 系统找不到指定的文件。

### 小结

- MXNet包括用户直接用来交互的前端和系统用来执行计算的后端。
- MXNet能够通过异步计算提升计算性能，
- 建议使用每个小批量训练或预测时至少使用一个同步函数，从而避免在短时间内将过多计算任务丢给后端。